Transfer learning between different admission types (EMERGENCY vs ELECTIVE)

In [1]:
from ast import literal_eval
from collections import Counter
import pandas as pd


admid_diagnosis_df = pd.read_csv("/home/wanxinli/EHR-OT/outputs/mimic/ADMID_DIAGNOSIS.csv", header=0, index_col=0,  converters={"ICD codes": literal_eval})
adm_types = list(admid_diagnosis_df['adm_type'])
print(Counter(adm_types))

Counter({'EMERGENCY': 42071, 'NEWBORN': 7863, 'ELECTIVE': 7706, 'URGENT': 1336})


In [2]:
import sys
sys.path.append("/home/wanxinli/EHR-OT/")

from mimic_common import *
from multiprocess import Pool
import os
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import time


/home/wanxinli/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
output_dir = os.path.join(os.path.expanduser("~"), f"EHR-OT/outputs/mimic")
print(f"Will save outputs to {output_dir}")

Will save outputs to /home/wanxinli/EHR-OT/outputs/mimic


In [4]:
"""
Train deep patient model and generate representations for targets and sources
"""

def custom_train_reps(target_features, source_features, n_components, pca_explain=False):
    """ 
    Customized training algorithm for generating target representations and source representations

    :param bool pca_explain: print the explained variance of each components
    
    :returns: target representations, source representations
    """
    source_pca = PCA(n_components=n_components)
    target_pca = PCA(n_components=n_components)
    target_reps = target_pca.fit_transform(target_features)
    source_reps = source_pca.fit_transform(source_features)

    if pca_explain:
        source_exp_var = source_pca.explained_variance_ratio_
        source_cum_sum_var = np.cumsum(source_exp_var)
        target_exp_var = target_pca.explained_variance_ratio_
        target_cum_sum_var = np.cumsum(target_exp_var)
        print("Cummulative variance explained by the source PCA is:", source_cum_sum_var)
        print("Cummulative variance explained by the target PCA is:", target_cum_sum_var)

    return target_reps, source_reps

In [5]:
def multi_proc_parallel(score_path, n_components, label_code, custom_train_reps, \
        male_count, female_count, iteration=20):
    """ 
    Code cannot be parallized when passing the dataframe (full_df) as a parameter
    Hence, cannot be put into mimic_common.py
    """
    
    p = Pool(10)

    # note: the following line cannnot be used for parallelization either
    # admid_diagnosis_df = pd.read_csv("../../outputs/mimic/ADMID_DIAGNOSIS.csv", index_col=0, header=0, converters={'ICD codes': literal_eval})

    def iteration_wrapper(iter):
        """ 
        Wrapper function for one iteration, returns result statistics, for parallel computing

        :param int iter: the current iteration
        """
        print(f"iteration: {iter}\n")
        cur_res = entire_proc_binary(n_components, "adm_type", "ELECTIVE",  "EMERGENCY", label_code, admid_diagnosis_df, custom_train_reps, 
                    male_count = male_count, female_count = female_count, transfer_score=True)
        
        return cur_res

    res = p.map(iteration_wrapper, np.arange(0, iteration, 1))
    res_df = pd.DataFrame(res, columns = ['target_accuracy', 'target_precision', 'target_recall', 'target_f1', \
                                          'source_accuracy', 'source_precision', 'source_recall', 'source_f1', \
                                            'trans_source_accuracy', 'trans_source_precision', 'trans_source_recall', 'trans_source_f1', 'transfer_score', 'original_score'])
    res_df.to_csv(score_path, index=False, header=True)
    return res



In [6]:
""" 
Run the entire proc for all response (i.e., label_code) 
Responses are selected by select_codes.ipynb and saved in ../../outputs/mimic/selected_summary_mimic.csv
"""

n_components = 20
male_count = 120
female_count = 50
label_code_path = os.path.join(output_dir, "selected_summary_mimic.csv")
label_code_df = pd.read_csv(label_code_path, header=0, index_col=None)
label_codes = list(label_code_df['ICD code'])[:1]
print("label_codes are:", label_codes)
for label_code in label_codes:
    start_time = time.time()
    print(f"label code {label_code} started")
    score_path = os.path.join(output_dir, f"exp6_{label_code}_score.csv")
    multi_proc_parallel(score_path, n_components, label_code, custom_train_reps, \
            male_count, female_count, iteration=100)
    end_time = time.time()
    print(f"runtime for {label_code} is: {end_time-start_time}")

label_codes are: ['008.45']
label code 008.45 started
iteration: 0
iteration: 3

iteration: 6
iteration: 12


iteration: 9


iteration: 15

iteration: 18

iteration: 21

iteration: 24

iteration: 27

iteration: 13

iteration: 10

iteration: 4

iteration: 7

iteration: 28

iteration: 19

iteration: 25

iteration: 22

iteration: 16

iteration: 1

iteration: 20

iteration: 14

iteration: 29

iteration: 5

iteration: 8

iteration: 23

iteration: 26

iteration: 17

iteration: 11

iteration: 2

iteration: 30

iteration: 33

iteration: 36

iteration: 39

iteration: 42

iteration: 45

iteration: 48

iteration: 51

iteration: 54

iteration: 57

iteration: 40

iteration: 37

iteration: 34

iteration: 31

iteration: 46

iteration: 43

iteration: 49

iteration: 52

iteration: 58

iteration: 55

iteration: 38

iteration: 41

iteration: 35

iteration: 44



/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iteration: 50

iteration: 47

iteration: 32

iteration: 53

iteration: 59

iteration: 56

iteration: 60

iteration: 63

iteration: 66

iteration: 69

iteration: 72

iteration: 75



/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iteration: 78

iteration: 81



...

iteration: 71

iteration: 68

iteration: 74

iteration: 65

iteration: 62



/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iteration: 77

iteration: 80

iteration: 86

iteration: 83

iteration: 89

iteration: 90

iteration: 93

iteration: 96

iteration: 99



/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iteration: 94

iteration: 91

iteration: 97

iteration: 92

iteration: 95

iteration: 98



In [ ]:
# label_code = "008.45"
# cur_res = entire_proc_binary(n_components, "adm_type",  "ELECTIVE", "EMERGENCY", label_code, admid_diagnosis_df, custom_train_reps, 
#                     male_count=male_count, female_count = female_count, transfer_score=True)

In [ ]:
# cur_res

(0.9775106630476929,
 0.2,
 0.038461538461538464,
 0.06451612903225806,
 0.9767176443734307,
 0.3333333333333333,
 0.02,
 0.03773584905660377,
 0.9769459027619265,
 0.4,
 0.02,
 0.03809523809523809,
 5.840246481991272)